In [1]:
# !pip install youtube-transcript-api transformers torch
import sys
!{sys.executable} -m pip install youtube-transcript-api transformers torch

In [2]:
# !pip install pytube
import sys
!{sys.executable} -m pip install pytube

In [3]:
!pip install beautifulsoup4 requests

In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import requests
from bs4 import BeautifulSoup
import re
import torch
import textwrap

# 파일 이름에서 쓸 수 없는 문자 제거
def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name)

# 유튜브 영상 ID 추출
def extract_video_id(url):
    match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
    if match:
        return match.group(1)
    else:
        raise ValueError("유효한 유튜브 URL이 아닙니다.")

# 유튜브 영상 제목 가져오기 (requests + BeautifulSoup)
def get_video_title(url):
    video_id = extract_video_id(url)
    clean_url = f"https://www.youtube.com/watch?v={video_id}"
    print(f"🎬 정제된 URL: {clean_url}")

    try:
        response = requests.get(clean_url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        raw_title = soup.title.string if soup.title else None
        if not raw_title:
            raise ValueError("title 태그 없음")

        cleaned = raw_title.replace("- YouTube", "").strip()
        return sanitize_filename(cleaned)
    except Exception as e:
        print(f"⚠️ 영상 제목을 불러오지 못했습니다. 오류: {e}")
        return f"제목없음_{video_id}"

# 자막 가져오기
def get_korean_transcript(video_id):
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    for transcript in transcript_list:
        if transcript.language_code.startswith("ko"):
            return transcript.fetch()
    for transcript in transcript_list:
        if transcript.is_translatable:
            try:
                return transcript.translate('ko').fetch()
            except Exception:
                continue
    raise ValueError("한국어 자막 또는 번역 가능한 자막을 찾을 수 없습니다.")

# 자막 병합
def merge_transcript(transcript):
    merged = []
    for entry in transcript:
        try:
            merged.append(entry['text'])  # dict 타입
        except (TypeError, KeyError):
            try:
                merged.append(entry.text)  # 객체 타입
            except AttributeError:
                merged.append(str(entry))  # fallback
    return " ".join(merged)

# 요약 함수
def summarize_kobart(text, tokenizer, model, max_input=1024):
    inputs = tokenizer.encode(text, return_tensors='pt', max_length=max_input, truncation=True)
    summary_ids = model.generate(inputs, max_length=256, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 단일 영상 요약 프로세스 (저장 없이 출력만)
def summarize_youtube_video_korean(url, tokenizer, model):
    print("="*80)
    print(f"[▶] 유튜브 URL: {url}")
    try:
        video_id = extract_video_id(url)
        video_title = get_video_title(url)
        print(f" → 제목: {video_title}")

        transcript = get_korean_transcript(video_id)
        full_text = merge_transcript(transcript)

        wrapped = textwrap.wrap(full_text, 800)
        summary_list = []
        for i, chunk in enumerate(wrapped):
            print(f"   → Chunk {i+1}/{len(wrapped)} 요약 중...")
            summary = summarize_kobart(chunk, tokenizer, model)
            summary_list.append(summary)

        final_summary = "\n".join(summary_list)
        print("\n💡 요약 결과:")
        print(final_summary)
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    print("="*80 + "\n")

# 여러 영상 처리
def summarize_multiple_videos(url_list):
    print("[KoBART 모델 로딩 중...]")
    tokenizer = PreTrainedTokenizerFast.from_pretrained("digit82/kobart-summarization")
    model = BartForConditionalGeneration.from_pretrained("digit82/kobart-summarization")

    for url in url_list:
        summarize_youtube_video_korean(url, tokenizer, model)

# 실행 예시
if __name__ == "__main__":
    urls = [
        "https://youtu.be/U8kW7KNrSuk?si=Jcq-_T3TuZ3CY33R&t=1563",
        "https://youtu.be/ppow1Afy31k?t=1265&si=AE-IE5KtLwfELe8U",
        # 여기에 다른 유튜브 URL 추가 가능
    ]
    summarize_multiple_videos(urls)

[KoBART 모델 로딩 중...]


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


[▶] 유튜브 URL: https://youtu.be/U8kW7KNrSuk?si=Jcq-_T3TuZ3CY33R&t=1563
🎬 정제된 URL: https://www.youtube.com/watch?v=U8kW7KNrSuk
 → 제목: ✨타로카드로 보는  “놀랍고도 예상치 못한 기회” #타로
   → Chunk 1/22 요약 중...
   → Chunk 2/22 요약 중...
   → Chunk 3/22 요약 중...
   → Chunk 4/22 요약 중...
   → Chunk 5/22 요약 중...
   → Chunk 6/22 요약 중...
   → Chunk 7/22 요약 중...
   → Chunk 8/22 요약 중...
   → Chunk 9/22 요약 중...
   → Chunk 10/22 요약 중...
   → Chunk 11/22 요약 중...
   → Chunk 12/22 요약 중...
   → Chunk 13/22 요약 중...
   → Chunk 14/22 요약 중...
   → Chunk 15/22 요약 중...
   → Chunk 16/22 요약 중...
   → Chunk 17/22 요약 중...
   → Chunk 18/22 요약 중...
   → Chunk 19/22 요약 중...
   → Chunk 20/22 요약 중...
   → Chunk 21/22 요약 중...
   → Chunk 22/22 요약 중...

💡 요약 결과:
오늘은 놀랍도 예상치 못한 기회에 대한 주제로 리딩을 준비했는데 1번 카드를 선택하신 분들은 혼자만의 시간을 즐기고 혼자만의 그 노력과 능력으로 무엇인가를 이뤄내는 분들로 보여지는데 이번만큼은 여러분들이 사람들의 도움을 기꺼이 받아들이게 된다면 커다란 성취를 얻을 수 있을 것이다.
여러분들이 실행뿐만 아니라 도움의 손길까지 있게 되다 보니까 작은 일로 오히려 남들의 도움을 통해 큰 성과를 이루게 되지 않을까 이렇게 보여지고 있다.
퀸 오브 펜타클 킹 오브 펜타클이 함께 나와 있는 걸로 봐서는 여러분들에게 도

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import requests
from bs4 import BeautifulSoup
import re
import torch
import textwrap
import os

# 파일 이름에서 쓸 수 없는 문자 제거
def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name)

# 유튜브 영상 ID 추출
def extract_video_id(url):
    match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
    if match:
        return match.group(1)
    else:
        raise ValueError("유효한 유튜브 URL이 아닙니다.")

# 유튜브 영상 제목 가져오기 (requests + BeautifulSoup)
def get_video_title(url):
    video_id = extract_video_id(url)
    clean_url = f"https://www.youtube.com/watch?v={video_id}"
    print(f"🎬 정제된 URL: {clean_url}")

    try:
        response = requests.get(clean_url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        raw_title = soup.title.string if soup.title else None
        if not raw_title:
            raise ValueError("title 태그 없음")

        # 유튜브 제목은 "영상 제목 - YouTube" 형태
        cleaned = raw_title.replace("- YouTube", "").strip()
        return sanitize_filename(cleaned)
    except Exception as e:
        print(f"⚠️ 영상 제목을 불러오지 못했습니다. 오류: {e}")
        return f"제목없음_{video_id}"

# 자막 가져오기
def get_korean_transcript(video_id):
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    for transcript in transcript_list:
        if transcript.language_code.startswith("ko"):
            return transcript.fetch()
    for transcript in transcript_list:
        if transcript.is_translatable:
            try:
                return transcript.translate('ko').fetch()
            except Exception:
                continue
    raise ValueError("한국어 자막 또는 번역 가능한 자막을 찾을 수 없습니다.")

# 자막 병합
def merge_transcript(transcript):
    merged = []
    for entry in transcript:
        try:
            merged.append(entry['text'])  # dict 타입
        except (TypeError, KeyError):
            try:
                merged.append(entry.text)  # 객체 타입
            except AttributeError:
                merged.append(str(entry))  # fallback
    return " ".join(merged)

# KoBART 요약
def summarize_kobart(text, tokenizer, model, max_input=1024):
    inputs = tokenizer.encode(text, return_tensors='pt', max_length=max_input, truncation=True)
    summary_ids = model.generate(inputs, max_length=256, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 전체 프로세스
def summarize_youtube_video_korean(url, output_dir="D:/pythondata"):
    print("[1] 영상 ID 추출")
    video_id = extract_video_id(url)

    print("[2] 영상 제목 가져오는 중...")
    video_title = get_video_title(url)
    print(f" → 제목: {video_title}")

    output_path = os.path.join(output_dir, f"{video_title}.txt")

    print("[3] 자막 가져오는 중...")
    transcript = get_korean_transcript(video_id)
    print(f"[3-1] 자막 문장 수: {len(transcript)}줄")

    full_text = merge_transcript(transcript)
    print(f"[4] 총 자막 길이: {len(full_text)}자")

    print("[5] KoBART 모델 불러오는 중...")
    tokenizer = PreTrainedTokenizerFast.from_pretrained("digit82/kobart-summarization")
    model = BartForConditionalGeneration.from_pretrained("digit82/kobart-summarization")

    print("[6] 자막 요약 중...")
    wrapped = textwrap.wrap(full_text, 800)
    summary_list = []
    for i, chunk in enumerate(wrapped):
        print(f" → Chunk {i+1}/{len(wrapped)} 요약 중...")
        summary = summarize_kobart(chunk, tokenizer, model)
        summary_list.append(summary)

    final_summary = "\n".join(summary_list)

    os.makedirs(output_dir, exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(final_summary)

    print(f"\n📄 요약 결과가 저장되었습니다: {output_path}")
    return final_summary

# 실행 예시
if __name__ == "__main__":
    youtube_url = "https://youtu.be/U8kW7KNrSuk?si=Jcq-_T3TuZ3CY33R&t=1563"
    try:
        summary = summarize_youtube_video_korean(youtube_url)
        print("\n💡 요약 결과:")
        print(summary)
    except Exception as e:
        print("오류 발생:", str(e))

[1] 영상 ID 추출
[2] 영상 제목 가져오는 중...
🎬 정제된 URL: https://www.youtube.com/watch?v=U8kW7KNrSuk
 → 제목: ✨타로카드로 보는  “놀랍고도 예상치 못한 기회” #타로
[3] 자막 가져오는 중...
[3-1] 자막 문장 수: 943줄
[4] 총 자막 길이: 16954자
[5] KoBART 모델 불러오는 중...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


[6] 자막 요약 중...
 → Chunk 1/22 요약 중...
 → Chunk 2/22 요약 중...
 → Chunk 3/22 요약 중...
 → Chunk 4/22 요약 중...
 → Chunk 5/22 요약 중...
 → Chunk 6/22 요약 중...
 → Chunk 7/22 요약 중...
 → Chunk 8/22 요약 중...
 → Chunk 9/22 요약 중...
 → Chunk 10/22 요약 중...
 → Chunk 11/22 요약 중...
 → Chunk 12/22 요약 중...
 → Chunk 13/22 요약 중...
 → Chunk 14/22 요약 중...
 → Chunk 15/22 요약 중...
 → Chunk 16/22 요약 중...
 → Chunk 17/22 요약 중...
 → Chunk 18/22 요약 중...
 → Chunk 19/22 요약 중...
 → Chunk 20/22 요약 중...
 → Chunk 21/22 요약 중...
 → Chunk 22/22 요약 중...

📄 요약 결과가 저장되었습니다: D:/pythondata\✨타로카드로 보는  “놀랍고도 예상치 못한 기회” #타로.txt

💡 요약 결과:
오늘은 놀랍도 예상치 못한 기회에 대한 주제로 리딩을 준비했는데 1번 카드를 선택하신 분들은 혼자만의 시간을 즐기고 혼자만의 그 노력과 능력으로 무엇인가를 이뤄내는 분들로 보여지는데 이번만큼은 여러분들이 사람들의 도움을 기꺼이 받아들이게 된다면 커다란 성취를 얻을 수 있을 것이다.
여러분들이 실행뿐만 아니라 도움의 손길까지 있게 되다 보니까 작은 일로 오히려 남들의 도움을 통해 큰 성과를 이루게 되지 않을까 이렇게 보여지고 있다.
퀸 오브 펜타클 킹 오브 펜타클이 함께 나와 있는 걸로 봐서는 여러분들에게 도움의 손길을 줬기 때문에 여러분들은 혼자만의 것이 아니고 베풀고 나눌 수 있게 되는 그런 마음을 가질 수 있게 된다고 이야기하고 있다.
이 스브 완드 카드가 나와 있는 걸로 봐서는 혼자만의 노력이 아니고 함께 이루어낸 공